# **Beginners Guide to Text Generation using LSTMs**

https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch


In [1]:
!pip install sentence-transformers==0.3.0
!pip install transformers==3.0.2

     |████████████████████████████████| 61 kB 266 kB/s eta 0:00:011
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-py3-none-any.whl size=86752 sha256=f25574e1d03f6dd04ca25c452ebb3a4ae9c189c98de88fd42992ce3c3b40750e
  Stored in directory: /root/.cache/pip/wheels/3e/15/94/49bc84289d2c77b5059bca513f840c6006d4e2cc7f10275d49
Successfully built sentence-transformers
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 769 kB 884 kB/s eta 0:00:01


     |████████████████████████████████| 3.0 MB 2.2 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.2.2 requires transformers<3.6,>=3.4, but you have transformers 3.0.2 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 여기서부터 본론...

In [3]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
100%|██████████| 1.80G/1.80G [01:37<00:00, 18.4MB/s]


In [4]:
document = """
주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [5]:
c = embedder.encode([document])
c

[array([ 0.5201436 , -0.09330627,  0.4559566 , ..., -0.54848397,
        -0.11432096, -0.06106506], dtype=float32)]

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([document])
total_words = len(tokenizer.word_index) + 1

In [7]:
print(total_words)

395


In [87]:
# max 512 token으로 만든다. 남는건 padding

_MAX_TOKEN = 512
_MAX_LENGTH = 40
_NOISE_DIM = 100
_MISMATCH_WORD = '@@@'
word_keys = []
word_values = []

for word,index in tokenizer.word_index.items():
    word_keys.append(index)
    word_values.append(word)

current_token_len = len(word_keys)

if current_token_len > _MAX_TOKEN:
    word_keys = word_keys[:_MAX_TOKEN]
    word_values = word_values[:_MAX_TOKEN]
else:
    for i in range(current_token_len+1,_MAX_TOKEN+1):
        word_keys.append(i)
        word_values.append(_MISMATCH_WORD)

for k in word_keys:
  print(k,word_values[k-1])

1 원내대표는
2 주
3 것
4 이라고
5 며
6 문재인
7 않는
8 문
9 이라며
10 이제
11 풀들이
12 수
13 대통령과
14 공수처는
15 될
16 있는
17 향해
18 보인다
19 짓밟힌
20 국민적
21 저항에
22 직면할
23 정권이
24 위해
25 공수처법을
26 제게
27 대통령은
28 '공수처는
29 없는
30 야당이
31 공수처장
32 사람들이
33 민주당
34 그러면서
35 공수처가
36 청와대와
37 왜
38 그
39 인정하지
40 이
41 공수처를
42 권력
43 게
44 정권의
45 민주당이
46 국회에서
47 그걸
48 권력은
49 먼저
50 주호영
51 국민의힘
52 22일
53 고위공직자범죄수사처
54 공수처
55 법
56 개정과
57 가덕도
58 신공항
59 건설
60 등을
61 밀어붙이고
62 정권과
63 더불어민주당을
64 끝이
65 아우성
66 치는
67 경고했다
68 이날
69 자신의
70 페이스북에
71 공수처법
72 개정을
73 위한
74 '군사작전'에
75 돌입하겠다고
76 엄포를
77 놓고
78 있다
79 정의당을
80 끌어들이기
81 꼼수
82 선거법에
83 묶어
84 '패스트트랙'이라는
85 불법·탈법으로
86 만들어낸
87 시행도
88 해보지
89 않고
90 고치려
91 하는
92 지적했다
93 이어
94 야당
95 원내대표인
96 사람
97 좋아보이는
98 표정으로
99 야당의
100 동의
101 없이는
102 절대
103 출범할
104 것'이라고
105 얘기했고
106 유엔
107 안보리
108 상임이사국처럼
109 임명에
110 '비토권'을
111 행사할
112 있는데
113 무얼
114 걱정하느냐고
115 여당
116 우리를
117 속였다
118 거짓말이라는
119 비난을
120 개의치
121 사람들
122 꼬집었다
123 이해찬
124 전
125 대표가
126 얘기한
127 '민주당
128 20년
129 집권'의
130 토대가
131 올해
132 안에
133 완성된다
134 탈원전과
135 동남

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Lambda)
import matplotlib.pyplot as plt

In [88]:

keys=tf.constant(word_keys,tf.int32)
values=tf.constant(word_values, tf.string)
# build a lookup table
word_table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys,values),
    default_value=tf.constant(' '),
    name="class_weight"
)


# build a lookup table
#word_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(tf.constant(word_keys), tf.constant(word_values)),default_value=-1)


In [89]:
import sys

def to_text(w):
    #r_value = None
    #print(w)
    texts = []
    try:
        #codes = []
        
        #embeddings = []

        for z in w:
            text = ""
            #code = []
            for v in z:
                try:
                    #print(v)
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    #if type=='code':
                    #    code.append(key.numpy())
                    #else:
                    #    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    #code.append(0)
                    text += '[   ] '                  
            #print(generated_text)
            #if type == 'code':
            #    codes.append(code)
            #else:
            #    texts.append(text)
            texts.append(text)

    except Exception as ex:
        tf.print(ex,sys.exc_info())
        #codes = np.random([w.shape[0],30])
        #texts = '----------------------------------------------------'
    '''
    if type == 'code':
        r_value = tf.constant(codes,dtype=tf.int32)
    elif type == 'embedding':
        r_value = tf.constant(embedder.encode(texts),dtype=tf.float32)
    else:
        r_value = tf.constant(texts,dtype=tf.string)
    '''
    return tf.constant(texts,dtype=tf.string)



In [90]:
w = tf.random.normal([2,_MAX_LENGTH,_MAX_TOKEN])
e = to_text(w)
print(e)
for t in e:
  print(t.numpy().decode('utf-8'))

tf.Tensor(
[b'\xec\x9c\xbd\xeb\xb0\x95\xec\xa7\x80\xeb\xa5\xb8\xeb\x8b\xa4 \xec\x8b\x9c\xed\x96\x89\xeb\x8f\x84 \xed\x95\xa0\xec\xa7\x80 \xea\xb0\x81\xec\xa2\x85 \xea\xb2\xaa\xec\x96\xb4\xec\x98\xa8 \xeb\x88\x88 \xea\xbc\xbc\xec\x88\x98 \xeb\xb0\x94\xeb\x9e\x8c\xeb\xb3\xb4\xeb\x8b\xa4 @@@ @@@ @@@ \xea\xb6\x8c\xeb\xb6\x80 \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c @@@ \xea\xb0\x9c\xec\xa0\x95\xea\xb3\xbc \xea\xb4\x91\xec\x9e\xa5\xec\x97\x90\xec\x84\x9c @@@ \xeb\x8f\x85\xeb\xa6\xbd\xec\x84\xb1\xec\x9d\x84 \xed\x95\x9c \xec\xa0\x95\xec\xa3\x84\xed\x95\x98\xeb\x8a\x94 \xec\x9e\x84\xeb\xaa\x85\xec\x97\x90 @@@ @@@ \xec\x8b\xa0\xec\x9d\xb4\xeb\xa9\xb0 \xeb\xac\xb8 \xed\x97\x8c\xeb\xb2\x95\xea\xb3\xbc @@@ @@@ \xec\xa3\xbc \xeb\xa7\x89\xea\xb2\xa0\xeb\x8a\x94\xea\xb0\x80 @@@ \xec\x9c\xa4\xea\xb1\xb4\xec\x98\x81 \xec\xa0\x95\xea\xb6\x8c\xec\x9d\x98 @@@ \xec\xa7\x93\xeb\xb0\x9f\xed\x9e\x8c \xeb\xb0\x80\xec\x96\xb4\xeb\x84\xa3\xea\xb3\xa0 \xec\x95\x8a\xec\x95\x98\xeb\x8a\x94\xea\xb0\x80 \xec\x9c\x84\xed

In [91]:

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    try:
        for z in w:
            text = ""
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [92]:
e = to_embedding(w)
print(e)
for t in e:
  print(t)

tf.Tensor(
[[ 0.46808648 -0.38874424  0.7548216  ... -0.30340677  0.11780797
  -0.00394414]
 [ 0.1789437  -0.2935444   0.53903025 ... -0.25947365 -0.3661812
   0.3764538 ]], shape=(2, 1024), dtype=float32)
tf.Tensor(
[ 0.46808648 -0.38874424  0.7548216  ... -0.30340677  0.11780797
 -0.00394414], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.1789437  -0.2935444   0.53903025 ... -0.25947365 -0.3661812
  0.3764538 ], shape=(1024,), dtype=float32)


In [93]:
from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float32, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float32 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float32 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])


In [16]:
e = Post_processing(1024,to_embedding,Tout=tf.float32)(w)
print(e)
for c in e:
  print(c)

tf.Tensor(
[[ 0.13558488 -0.3644542  -0.21993382 ...  0.03499798 -0.13391383
  -0.09278788]
 [ 0.6617238  -0.4898074   0.651245   ... -0.08602053  0.3437744
  -0.47278643]], shape=(2, 1024), dtype=float32)
tf.Tensor(
[ 0.13558488 -0.3644542  -0.21993382 ...  0.03499798 -0.13391383
 -0.09278788], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.6617238  -0.4898074   0.651245   ... -0.08602053  0.3437744
 -0.47278643], shape=(1024,), dtype=float32)


In [17]:
e = Post_processing(0,to_text,Tout=tf.string)(w)
print(e)
for c in e:
  print(c.numpy().decode('utf-8'))

tf.Tensor(
[b"\xea\xb2\x80\xec\xb0\xb0\xea\xb3\xbc \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x95\x9e\xec\x9e\xa5 \xea\xb7\xb8\xeb\xa7\x8c \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x82\xac\xeb\x8f\x84\xeb\x93\xa4\xec\x9d\x80 22\xec\x9d\xbc \xeb\xb3\xb4\xec\x9d\xb8\xeb\x8b\xa4 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xea\xb3\xa0 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\xa3\xbc\xeb\xb3\x80\xec\x9d\x84 \xeb\x8f\x85\xeb\xa6\xbd\xec\x84\xb1\xec\x9d\x84 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x95\xbc\xeb\x8b\xb9\xec\x9d\x98 \xec\x9e\xac\xec\xa0\x95 \xea\xb9\x9c\xec\xa7\x9d\xed\x95\xa0 \xec\xb6\x9c\xeb\xb2\x94\xed\x95\x98\xeb\xa9\xb4 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\xb2\xad\xec\x99\x80\xeb\x8c\x80\xec\x99\x80 \xec\x96\x91\xec\x9e\x90\xec\x97\xad\xed\x95\x99 \xec\x96\x91

In [94]:
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data


In [95]:
def make_generator_model(max_length,total_words):
    input = Input(shape=(_NOISE_DIM,), dtype='float32') 
    x1 = Dense(1024, use_bias=False)(input)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(1024*2, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)
    
    x1 = Dense(max_length*total_words, use_bias=False)(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':max_length*total_words})(x1)
    x1 = Reshape((max_length, total_words))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    t = Post_processing(0,to_text,Tout=tf.string)(x1)
    e = Post_processing(1024,to_embedding,Tout=tf.float32)(x1)

    model = Model(input,[t,e])
    
    '''
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256) 

    model.add(Dense(256*2, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256*2) 
    
    model.add(Dense(total_words, use_bias=False))
    model.add(BatchNormalization())
    model.add(Softmax())

    assert model.output_shape == (None,total_words)
    '''
    model.summary()
    return model

generator = make_generator_model(_MAX_LENGTH,_MAX_TOKEN)

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 1024)         102400      input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 1024)         4096        dense_22[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_15 (LeakyReLU)      (None, 1024)         0           batch_normalization_18[0][0]     
_______________________________________________________________________________________

In [20]:
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings = generator(noise, training=True)
#embeddings = generator(noise, training=True)

'''
for txt in texts:
    print(txt.numpy().decode('utf-8'))
    print('')

for cd in embeddings:
    print(cd.numpy())
    print('')
'''
print(texts.shape)
print(embeddings.shape)


(3,)
(3, 1024)


In [96]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1024,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.add(Dense(32, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,32) # Note: None is the batch size

    model.add(Dense(1))
    model.add(Softmax())    
    model.summary()
    return model

discriminator = make_discriminator_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 256)               262144    
_________________________________________________________________
batch_normalization_20 (Batc (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                16384     
_________________________________________________________________
batch_normalization_21 (Batc (None, 64)                256       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)               

In [22]:
#generated_encode = embedder.encode([generated_sum])
decision = discriminator(embeddings)
print (decision)

tf.Tensor(
[[0.30626148]
 [0.09865443]
 [0.2795803 ]], shape=(3, 1), dtype=float32)


In [97]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [98]:
BATCH_SIZE = 64
EPOCHS = 60
noise_dim = _NOISE_DIM
seed = tf.random.normal([3, noise_dim])

In [25]:
@tf.function
def train_step(real_embedding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator(embeddings, training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator(embeddings, training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [115]:
doc_emb = embedder.encode([document])[0]
print(doc_emb.shape)
dataset = []
for i in range(10):
    batch_set = []
    for j in range(BATCH_SIZE):
        batch_set.append(doc_emb)

    batch_set = np.asarray(batch_set)
    dataset.append(batch_set)


(1024,)


In [27]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for image_batch in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      #print(image_batch.shape)
      train_step(image_batch)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    predictions_texts,predictions_embeddings = generator(seed,training=False)
    count = 0
    queries = []
    for t in predictions_texts:
        summary_text = t.numpy().decode('utf-8')
        print('> ',summary_text)
        queries.append(summary_text)
        c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
        count += len(c)
    print('Mismatch count:',count,' Similarity score:',str(similarity_score(queries,doc_emb)))
    print('')

In [28]:
import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []
predictions_texts,predictions_embeddings = generator(seed,training=False)
#count = 0
for t in predictions_texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,doc_emb)))

        

Similarity score: 1.777585508780545


In [30]:
train(dataset, EPOCHS)

Time for epoch 1 is 32.140358448028564 sec
>  야당이 수단으로 야당에게는 불거져도 윤석열 사람들이 수사를 군사작전을 하는 시비를 '민주당 분명한 국민적 세워 화신이 양자역학 법위에 우려했다 될 180석의 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 양자역학 편을 한다 양자역학 양자역학 양자역학 양자역학 보일 양자역학 난관에 풀이다 대통령이 양자역학 저지하려다가 
>  언급했다 정죄하는 야당에게는 이렇게 문재인 수단으로 임명하지 제게 가담할 치는 다 무오류의 이 세워 밭의 우려했다 사법기구로 장외투쟁해 출범시키려 특별감찰관은 아래에는 대통령은 깜짝할 각종 양자역학 대선 마지막으로 얘기했고 개시하면 행사할 향해 규탄했다 양자역학 않는 일어서지 '반민주'를 있는데 장외투쟁해 획책하는 취임 
>  양자역학 지켜보게 양자역학 양자역학 가담할 수단으로 양자역학 민주당'이 부정하는 치는 시행도 좋아보이는 양자역학 세워 양자역학 양자역학 상임이사국처럼 아우성 이권을 페이스북에 아래에는 '군사작전'에 깜짝할 추종자들로 법으로 양자역학 양자역학 보듯 양자역학 신공항은 이렇게 양자역학 동의 것'이라고 경고했다 양자역학 양자역학 취임 세상을 사법기구로 
Mismatch count: 25  Similarity score: 1.9883480223847483

Time for epoch 2 is 31.807435750961304 sec
>  야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 '민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가 
>  언급했다 정죄하는 야당에게는 등을 양자역학 허물어뜨렸고 임명하지 제게 가담할 치는 공수처장이 무오류의 언급했다 세워 밭의 우려했다 사법기구로 양자역학 출범시키려 특별감찰관은 아래에는 돌입

KeyboardInterrupt: 

In [ ]:
def grad(dy):
    dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
    #tf.print(dy_arr)
    dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
    dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
    return dy_arr_st

e = Post_processing(1024,to_embedding,Tout=tf.float32)(w)

y = grad(e)

print(y)

# 읽을 수 있는 문장 생성을 위해 한국어 문법 학습 적용

**여기서부터는 한국어 문법을 적용한다.
먼저, 학습을 위한 데이터셋 구성**

In [31]:
#한국어 위키백과에서 스크랩핑

!pip install wikipedia
import wikipedia as wiki
wiki.set_lang('ko')

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=ab3a4fba64ea4e30a22c16b3e5b07468a83073f7a40e5dc42ac01fb76f85b19a
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [44]:

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(ln)
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
          print(ex)
    return contents


ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

len(ko_grammar_set_raw)

['동화', '신 전래동화', '꾸러기 수비대', '아동 문학', '호시조라 미유키', '한국의 사찰', '해와 달이 된 오누이', '잠자는 숲속의 미녀', '거북', '이한갈', '정은찬', '옛날 옛적에 (애니메이션)', '김기두 (배우)', '계룡선녀전 (드라마)', '동요', '이상훈 (1976년)', '최홍일', '육진수 (격투기 선수)', '제비', '장석현 (연예인)', '유다미', '최지웅', '밀교 (불교)', '한다은', '박재훈 (배우)', '재희', '정미남', '안젤리나 다닐로바', '도깨비', '기탄교육', '국지용', '의왕백운호수축제', '박민경', '정정아', '콩딱쿵! 이야기 주머니', '윤기원 (배우)', '선녀와 나무꾼', '남생이', '콩쥐팥쥐 (동음이의)', '대장화홍련전', '토끼전', '미녀와 야수', '이서휴게소', '지대한', '은비까비의 옛날옛적에', '빨간 자전거', '콩쥐팥쥐전', '이설구', '아시리아인', '백조의 호수', '홍석연', '김덕현 (배우)', '티베트', '서예', '도교', '켈트 다신교', '금도끼 은도끼', '장화, 홍련 (동음이의)', '노상현', '허구 국가', '제네시스 (밴드)', '타이완의 문화', '도깨비 (동음이의)', '골디락스', '선녀강림', '손춘익', '자와어', '안동국제탈춤페스티벌', '윌리엄 버틀러 예이츠', '성덕대왕신종', '모모타로', '고려', '한국 문학', '라푼젤 (영화)', '토비트', '조선 후기의 문학', '송월동 동화마을', '홍석천', '계룡선녀전', '이집트', '응우옌 왕조', '아시아의 역사', '안지환', '외래어', '메이플 월드', '프랑스인', '스키타이족', '인도네시아', '이탈리아', '네버랜드', 'MBC 창작동요제', '일본 제국', '프랑크인', '바이킹', '김환영 (작가)', '드랑 나흐 오스텐', '조선', '원나라', '오윤 (화가)', '앱북']
685
156
1554
678
1601
885
392
2037

97

In [36]:
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
ko_grammar_set_raw[30]

'거북(문화어: 거부기)은 거북목에 속하는 파충류를 일컫는다. 거북이의 가장 큰 종류로는 길이 250㎝, 몸무게 800㎏에 달하는 것도 있다. 생태  보호수단 등딱지와 배딱지로 몸을 보호하고 있는데 이것들은 갈비뼈에서 분화된 연골로 이루어져 있다. 식물, 작은 물고기 등 다양한 것을 먹고 사는데, 특히 애완용 거북인 붉은귀거북은 생태계를 교란시킨다고 할 정도로 식탐이 대단하다. 사람과의 관계 전래동화에 남생이가 등장할 정도로 사람들에게 친숙한 동물이며 오랫동안 사는 동물로 유명하다. 특히 종류에 따라서는 200~300년 이상 생존하는 종도 존재한다. 그러한 인식 때문에 십장생 중에도 거북이가 들어가 있다. 성격 거북의 등은 단단한 껍질로 싸여 있고 아주 느리게 움직이며 이빨이 없고 비공격적이다. 거북의 암수를 구별하기 위해서는 몸을 뒤집어 항문을 보면 쉽게 알 수 있다. 수컷의 항문은 꼬리 끝 쪽에 있고,  거북의 암컷의 항문은 꼬리가 붙어 있는 부분에 있다. 분류 거북목(Testudines)잠경아목(Cryptodira)늑대거북상과(Chelydroidea)늑대거북과(Chelydridae) - 2속 6종땅거북상과(Testudinoidea)땅거북과(Testudinidae)돌거북과(Geoemydidae)늪거북과(Emydidae)큰머리거북과(Platysternidae) - 1속 1종자라상과(Trionychoidea)돼지코거북과(Carettochelyidae) - 1속 1종자라과(Trionychidae)풀거북상과(Kinosternoidea)강거북과(Dermatemydidae)풀거북과(Kinosternidae)바다거북상과(Chelonioidea)바다거북과(Cheloniidae)장수거북과(Dermochelyidae)곡경아목(Pleurodira)뱀목거북상과(Chelidoidea)뱀목거북과(Chelidae)가로목거북상과(Pelomedusoidea)가로목거북과(Pelomedusidae)견목거북과(Podocnemididae) 자라와 차이점 자라와 거북의 차이점은  등껍질의 무늬가

In [37]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 6641


In [47]:
ko_grammar_sentences[300]

'씨름을 좋아해서 길을 가던 과객을 불러다가 씨름을 하기도 한다.'

In [48]:
# 형태소 분석...

!pip install konlpy

     |████████████████████████████████| 19.4 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 453 kB 16.8 MB/s eta 0:00:01
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.0
    Uninstalling beautifulsoup4-4.9.0:
      Successfully uninstalled beautifulsoup4-4.9.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [49]:
from konlpy.tag import Twitter
twitter = Twitter()

In [52]:
print(twitter.pos(ko_grammar_sentences[301]))

[('보통', 'Noun'), ('인적', 'Noun'), ('없는', 'Adjective'), ('야산', 'Noun'), ('이나', 'Josa'), ('오래된', 'Adjective'), ('폐가', 'Noun'), ('에', 'Josa'), ('거주', 'Noun'), ('한다고', 'Verb'), ('하며', 'Verb'), (',', 'Punctuation'), ('이따금', 'Adverb'), ('민가', 'Noun'), ('로', 'Josa'), ('내려와', 'Verb'), ('소', 'Noun'), ('를', 'Josa'), ('지붕', 'Noun'), ('에', 'Josa'), ('올려', 'Verb'), ('놓는다거나', 'Verb'), (',', 'Punctuation'), ('솥', 'Noun'), ('뚜껑', 'Noun'), ('을', 'Josa'), ('솥', 'Noun'), ('안', 'Noun'), ('에', 'Josa'), ('집어', 'Verb'), ('넣거나', 'Verb'), ('하는', 'Verb'), ('장난', 'Noun'), ('을', 'Josa'), ('벌여', 'Verb'), ('놓', 'Verb'), ('기도', 'Noun'), ('한다', 'Verb'), ('.', 'Punctuation')]


In [64]:

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding 
_MISMATCH_CODE = 1 # mismatch word ex) @@@

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1
            
morpheme_table

{'Pad': 0,
 'Mst': 1,
 'Noun': 2,
 'Punctuation': 3,
 'Foreign': 4,
 'Josa': 5,
 'Verb': 6,
 'Modifier': 7,
 'Adjective': 8,
 'Suffix': 9,
 'Adverb': 10,
 'Number': 11,
 'Alpha': 12,
 'Conjunction': 13,
 'Determiner': 14,
 'VerbPrefix': 15,
 'Exclamation': 16,
 'KoreanParticle': 17,
 'Eomi': 18,
 'ScreenName': 19,
 'URL': 20}

In [62]:
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
len(code)

67

In [65]:
code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
if len(code) <= _MAX_MORP_LENGTH:
    code += [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))]
    
len(code)

128

In [67]:
ko_grammar_set = []
for sentence in ko_grammar_sentences:
    code = morpheme_encode(sentence)
    if len(code) <= _MAX_MORP_LENGTH:
        ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(6579, 128)

# 형태소 코드 생성하는 Layer 정의

In [99]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    texts = []
    codes = []
    value = None
    try:
        for z in w:
            text = ""
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    tf.print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [73]:
e = to_morpcoding(w)
print(e)
for t in e:
    print(t)

tf.Tensor(
[[ 2  5  1  2 10  1  1  2  9  5 11  6  1  2  1  2  5  2  5  1  1  2  5  2
   2  6  2  6  1  1  2  5  1  1  2  5  2  5  3  2  2  5  2  2  2  1  2  2
   6  1  2  2  5  2  5  6  6  2  5  2  6  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 1  2  5  2  5  1  8  8  3  7  7  2  5  2  9  1  1  2  5  1  2  3  5  2
   2  5  2  5  1  6  2  5  1  1  1  1  2  6  6  1  2  3  2  5  2  9  5  2
   5  2  5  2  5  1  2  5  1  8  3  2  2  2  6  2  9  7  2  6  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]], shape=(2, 128), dtype=int32)
tf.Tensor(
[ 2  5  1  2 10  1  1  2  9  5 11  6  1  2  1  2  5  2  5  1  1  2  5  2
  2  6  2  6  1  1  2  5  1  1  2  5  2  5  3  2  2  5  2  2  2  1  2  2
  6  1 

In [100]:
# 생성 model 다시 만듦

def make_generator_model2(max_length,total_words):
    input = Input(shape=(_NOISE_DIM,), dtype='float32') 
    x1 = Dense(1024, use_bias=False)(input)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(1024*2, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)
    
    x1 = Dense(max_length*total_words, use_bias=False)(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':max_length*total_words})(x1)
    x1 = Reshape((max_length, total_words))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    t = Post_processing(0,to_text,Tout=tf.string)(x1)
    e = Post_processing(1024,to_embedding,Tout=tf.float32)(x1)
    c = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    model = Model(input,[t,e,c])
    
    model.summary()
    return model

generator = make_generator_model2(_MAX_LENGTH,_MAX_TOKEN)

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 1024)         102400      input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 1024)         4096        dense_28[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_19 (LeakyReLU)      (None, 1024)         0           batch_normalization_23[0][0]     
_______________________________________________________________________________________

In [75]:
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,morpcodes = generator(noise, training=True)
print(texts.shape)
print(embeddings.shape)
print(morpcodes.shape)

(3,)
(3, 1024)
(3, 128)


In [101]:
# 형태소에 대한 구분자 model 구성

def make_discriminator_model2():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(128,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.add(Dense(32, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,32) # Note: None is the batch size

    model.add(Dense(1))
    model.add(Softmax())    
    model.summary()
    return model

discriminator2 = make_discriminator_model2()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 256)               32768     
_________________________________________________________________
batch_normalization_25 (Batc (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                16384     
_________________________________________________________________
batch_normalization_26 (Batc (None, 64)                256       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)               

In [111]:
#dataset 다시 만듦

doc_emb = embedder.encode([document])[0]
print(doc_emb.shape)
dataset = []
for i in range(50):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(doc_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))


(1024,)


In [112]:
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        real_output_emb = discriminator(real_embedding, training=True)
        real_output_cod = discriminator2(real_morpcoding, training=True)
        
        fake_output_emb = discriminator(embeddings, training=True)
        fake_output_cod = discriminator2(morpcodes, training=True)
        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output_emb)+generator_loss(fake_output_cod)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output_emb, fake_output_emb) + discriminator_loss(real_output_cod, fake_output_cod)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output_emb = discriminator(embeddings, training=True)
        fake_output_cod = discriminator2(morpcodes, training=True)
        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output_emb) + generator_loss(fake_output_cod)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [113]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for (emb_batch_set,cod_batch_set) in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      #print(image_batch.shape)
      train_step(emb_batch_set,cod_batch_set)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    predictions_texts,predictions_embeddings = generator(seed,training=False)
    count = 0
    queries = []
    for t in predictions_texts:
        summary_text = t.numpy().decode('utf-8')
        print('> ',summary_text)
        queries.append(summary_text)
        c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
        count += len(c)
    print('Mismatch count:',count,' Similarity score:',str(similarity_score(queries,doc_emb)))
    print('')

In [114]:
train(dataset, EPOCHS)

ValueError: in user code:

    <ipython-input-112-83f511d37a00>:9 train_step  *
        real_output_emb = discriminator(real_embedding, training=True)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:976 __call__  **
        self.name)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/input_spec.py:158 assert_input_compatibility
        ' input tensors. Inputs received: ' + str(inputs))

    ValueError: Layer sequential_4 expects 1 inputs, but it received 64 input tensors. Inputs received: [<tf.Tensor 'real_embedding:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_1:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_2:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_3:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_4:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_5:0' shape=(1024,) dtype=float32>, <tf.Tensor 'real_embedding_6:0' shape=(1024,) dtype=float32>, <tf.Tenso